# Sieve of Randall

## Introduction

![alt text](./img/crypto.jpg "Introduction")

## Abstract

Last time I spoke with Randall, he was quite interested in a particular type of machines that
initially were designed to solve a problem and while doing so they were catalyzing its own
construction. He was pretty convinced that, as long as enough energy was around, any solvable
problem could be solved in that way. I couldn't find flaws in his logic, and we pursued an
exploration.

In fact, he pointed out that the machine will create (or better said, discover) new problems
on it’s way and solve them with the sole purpose of collecting and storing energy. And again,
he cleverly made a guess that different components of the machine will establish a symbiotic
relation to parallelize the resolution of the problems and to reuse previous solutions when
necessary.

That’s how the concept of RIST was born, which was the most natural way to study the behavior
of different parts of the machine over time. Something overlooked back then was the possibility
that the machine became aware of itself, in the sense that a RIST will develop enough
computing power to formulate a question about its nature of existence.

Randall once joked about that. “Ha! What if the machine now knows that something exists?
Is that a bug?”. And yes, consciousness was initially treated as a bug and it was used as
a stopping criteria for whatever problem was being solved.

But as a good friend of mine always says, a bug, it’s a feature. So we opened a new branch where
we let consciousness develop freely in the machine whenever it was possible.

Among other things, we found that the development of consciousness was quite different on
each iteration, depending on the initial problem. But regardless of that genesis trigger,
we found a set of problems that appeared over and over on every conscious instance of
the machine that we created. We didn’t probe it, but we had a good intuition that the
discovery of those problems is a necessary condition for consciousness to exist.

Since we were skipping our arts class at that time, we couldn’t think of a more creative name
than “Necessary Problems” for them. It was later changed to “Natural Problems” because no one
worked on a proof for that yet. Not even the machine $\zeta$

And here it is Randy, look at what you did !

## Prime Sieve Automata (PSA)

### Handcrafting the logic

Let's consider a language with an alphabet of "1" and "0" where natural numbers are expressed in the
base-2 numeral system. For example, 2 it's represented as "10" and 11 as "1011". The automata
we'll build is going to read the strings starting at the right most bit (least significant bit)
of the string.

In general, the following function converts a natural number into the binary
representation we'll work with.

In [35]:
def get_binary_input(number):
    binary = "{0:0128b}".format(number)
    stack = [c for c in binary[binary.find('1'):][::-1]]

    return stack

for i in range(1, 12):
    print("[+] binary stack for n =", i, get_binary_input(i))

[+] binary stack for n = 1 ['1']
[+] binary stack for n = 2 ['0', '1']
[+] binary stack for n = 3 ['1', '1']
[+] binary stack for n = 4 ['0', '0', '1']
[+] binary stack for n = 5 ['1', '0', '1']
[+] binary stack for n = 6 ['0', '1', '1']
[+] binary stack for n = 7 ['1', '1', '1']
[+] binary stack for n = 8 ['0', '0', '0', '1']
[+] binary stack for n = 9 ['1', '0', '0', '1']
[+] binary stack for n = 10 ['0', '1', '0', '1']
[+] binary stack for n = 11 ['1', '1', '0', '1']


Now let's build a finite automata which is going to accept binary strings of prime numbers until 11.
A straightforward way would be to create a single final state ($ ip_0 $) and connect the
sequence of ones and zeroes from the initial state ($ is_0 $) for each prime.

In [36]:
from zephod.finite import *

initial, final = State("is0"), State("ip0")

delta = FADelta()

# number 2
delta.add(initial, State("z0"), {"0"})
delta.add(State("z0"), final, {"1"})

# number 3
delta.add(initial, State("z1"), {"1"})
delta.add(State("z1"), final, {"1"})

# number 5
delta.add(initial, State("z2"), {"1"})
delta.add(State("z2"), State("z3"), {"0"})
delta.add(State("z3"), final, {"1"})

# number 7
delta.add(initial, State("z4"), {"1"})
delta.add(State("z4"), State("z5"), {"1"})
delta.add(State("z5"), final, {"1"})

# number 11
delta.add(initial, State("z6"), {"1"})
delta.add(State("z6"), State("z7"), {"1"})
delta.add(State("z7"), State("z8"), {"0"})
delta.add(State("z8"), final, {"1"})

eleven_sieve = FiniteAutomata(transition=delta, initial=initial, final={final})

print("[+] the eleven prime sieve")
print(eleven_sieve)

from utils.plotter import *
from IPython.display import IFrame

print("\n[+] automata plotter")

AutomataPlotter.tikz(eleven_sieve, filename="eleven_sieve", output="./img/")
IFrame("./img/eleven_sieve.pdf", width=900, height=400)

[+] the eleven prime sieve
states = {z3, z6, z1, z5, z7, z2, z0, z8, z4, ip0, is0} ; alphabet = {'0', '1'}
is0 (0) -> {z0}
is0 (1) -> {z4, z2, z6, z1}
z0 (1) -> {ip0}
z1 (1) -> {ip0}
z2 (0) -> {z3}
z3 (1) -> {ip0}
z4 (1) -> {z5}
z5 (1) -> {ip0}
z6 (1) -> {z7}
z7 (0) -> {z8}
z8 (1) -> {ip0}
initial = is0 ; final = {ip0}

[+] automata plotter


The resulting automata is a non deterministic (NFDA), since on the first state there are several
transitions with the same symbol ("1") going to several others.

We can use a prime sieve to verify that the automata works as expected.

In [37]:
from sympy import sieve

for i in range(2, 12):
    accepted = eleven_sieve.read(get_binary_input(i))
    if i in sieve:
        print("[+] checking prime number n =", i, "accepted =", accepted)
        assert accepted
    else:
        print("[+] checking composite number n =", i, "accepted =", accepted)
        assert not accepted

[+] checking prime number n = 2 accepted = True
[+] checking prime number n = 3 accepted = True
[+] checking composite number n = 4 accepted = False
[+] checking prime number n = 5 accepted = True
[+] checking composite number n = 6 accepted = False
[+] checking prime number n = 7 accepted = True
[+] checking composite number n = 8 accepted = False
[+] checking composite number n = 9 accepted = False
[+] checking composite number n = 10 accepted = False
[+] checking prime number n = 11 accepted = True


Given the construction method, what this automata is pretty much doing is a linear search into the
prime numbers that are encoded on its transition function. It will try each one of the 4 branches
that are coming out from $is_0$ until it gets to the final state, or until it rejects
the string for composite numbers.

You can also debug the evolution of the automata for a given string, along with the different paths
that is taking when is non deterministic.

In [38]:
eleven_sieve.debug(get_binary_input(11))

-------------------------------------------------------------------------------

initial (is0)         final {ip0}                                                                                       

[C0] 1 1 0 1               is0                           

-------------------------------------------------------------------------------
[C0] 1 1 0 1               [is0, 1(0)] to [z4, 1(1)]     
[C0] 1 1 0 1               [is0, 1(0)] to [z2, 1(1)]     
[C0] 1 1 0 1               [is0, 1(0)] to [z6, 1(1)]     
[C0] 1 1 0 1               [is0, 1(0)] to [z1, 1(1)]     
-------------------------------------------------------------------------------
[C0] 1 1 0 1               [z4, 1(1)] to [z5, 0(2)]      
[C0] 1 1 0 1               [z6, 1(1)] to [z7, 0(2)]      
[C0] 1 1 0 1               [z1, 1(1)] to [ip0, 0(2)]     
-------------------------------------------------------------------------------
[C0] 1 1 0 1               [z7, 0(2)] to [z8, 1(3)]      
----------------------------------

As a first step, we can convert this NFDA into a deterministic one, which is going to result in a
DFA where each state will have at most 2 transitions. The efficiency to determine if a number
is prime or not it will be much better on this case.

In [39]:
print("\n[+] deterministic FA eleven sieve")

deterministic_eleven_sieve = eleven_sieve.get_deterministic_automata()

AutomataPlotter.tikz(deterministic_eleven_sieve, filename="deterministic_eleven_sieve", output="./img/")
IFrame("./img/deterministic_eleven_sieve.pdf", width=900, height=300)


[+] deterministic FA eleven sieve


Much better and cleaner. We can further minimize the automata to reduce the number of states.

In [40]:
print("\n[+] deterministic FA eleven sieve")

minimal_eleven_sieve = deterministic_eleven_sieve.minimal()

AutomataPlotter.tikz(minimal_eleven_sieve, filename="minimal_eleven_sieve", output="./img/")
IFrame("./img/minimal_eleven_sieve.pdf", width=900, height=300)

for i in range(2, 12):
    accepted = minimal_eleven_sieve.read(get_binary_input(i))
    if i in sieve:
        print("[+] checking prime number (minimal DFA) n =", i, "accepted =", accepted)
        assert accepted
    else:
        print("[+] checking composite number (minimal DFA) n =", i, "accepted =", accepted)
        assert not accepted


[+] deterministic FA eleven sieve
[+] checking prime number (minimal DFA) n = 2 accepted = True
[+] checking prime number (minimal DFA) n = 3 accepted = True
[+] checking composite number (minimal DFA) n = 4 accepted = False
[+] checking prime number (minimal DFA) n = 5 accepted = True
[+] checking composite number (minimal DFA) n = 6 accepted = False
[+] checking prime number (minimal DFA) n = 7 accepted = True
[+] checking composite number (minimal DFA) n = 8 accepted = False
[+] checking composite number (minimal DFA) n = 9 accepted = False
[+] checking composite number (minimal DFA) n = 10 accepted = False
[+] checking prime number (minimal DFA) n = 11 accepted = True


It's also possible to get a grammar out of the FDA, which is going to generate the prime
numbers we encoded on it.

In [41]:
from utils.language.grammar import *

eleven_sieve_grammar = Grammar.build_from_finite_automata(minimal_eleven_sieve)

print("[+] eleven sieve grammar")
print(eleven_sieve_grammar)

print("\n[+] prime generation")
for each in eleven_sieve_grammar.enumerate(length=4):
    print("[+] number generated", int(each[::-1], 2))

[+] eleven sieve grammar
T = {'0', '1'} ; N = {'Q', 'R', 'P'}
S -> 0P
S -> 1R
R -> 0P
R -> 1
R -> 1Q
Q -> 0P
Q -> 1
P -> 1

[+] prime generation
[+] number generated 2
[+] number generated 3
[+] number generated 7
[+] number generated 5
[+] number generated 11


### Moving it forward

What if we keep applying this process for larger numbers? In fact, we could keep reusing previous
minimized DFAs iteratively while adding new primes to its structure.

Let's do it for all natural numbers up to $2^7$. We can easily automate the construction of the
automata with the following methods using our external sieve to find prime numbers.

In [42]:
class PrimeBuilder:
    def __init__(self, final_state):
        self.counter = 0
        self.final_state = final_state

    def get_new_state(self):
        self.counter += 1
        return State("zp" + str(self.counter))

    def state_factorization(self, initial_state, transition, n):
        if n in sieve:
            stack, state = get_binary_input(n), initial_state
            for ic, b in enumerate(stack):
                current_state = self.get_new_state()

                if ic == 0:
                    transition.add(state, current_state, b)
                elif ic == len(stack) - 1:
                    transition.add(state, self.final_state, b)
                else:
                    transition.add(state, current_state, b)

                state = current_state

    def build(self, initial_state, transition, lower, higher):
        for lm in range(lower, higher):
            self.state_factorization(initial_state, transition, lm)

# get initial state from the previous automata
initial = deterministic_eleven_sieve.initial
final = deterministic_eleven_sieve.final.copy()

prime_state = State("ip0")
final.add(prime_state)

delta = copy.deepcopy(deterministic_eleven_sieve.transition)

prime_builder = PrimeBuilder(final_state=prime_state)

# add number from 11 to 64 (7 bits)
prime_builder.build(initial, delta, 12, 129)

# build the new automata with the additional primes
nfda_seven_sieve = FiniteAutomata(transition=delta, initial=initial, final=final)

print("\n[+] seven bit NFDA  sieve")

AutomataPlotter.tikz(nfda_seven_sieve, filename="nfda_seven_sieve", output="./img/")
IFrame("./img/nfda_seven_sieve.pdf", width=900, height=1000)


[+] seven bit NFDA  sieve


Now we can apply a NFDA to an DFA conversion, followed by a minimization.

In [43]:
print("\n[+] deterministic DFA 7 bits sieve")

dfa_seven_sieve = nfda_seven_sieve.get_deterministic_automata()

AutomataPlotter.tikz(dfa_seven_sieve, filename="dfa_seven_sieve", output="./img/")
IFrame("./img/dfa_seven_sieve.pdf", width=900, height=500)

print("\n[+] minimal DFA 7 bits sieve")

minimal_seven_sieve = dfa_seven_sieve.minimal()

AutomataPlotter.tikz(minimal_seven_sieve, filename="minimal_seven_sieve", output="./img/")
IFrame("./img/minimal_seven_sieve.pdf", width=900, height=500)


[+] deterministic DFA 7 bits sieve

[+] minimal DFA 7 bits sieve


Let's do some testing and generate 7 $bits$ primes to make sure everything is good.

In [44]:
seven_bits_sieve_grammar = Grammar.build_from_finite_automata(minimal_seven_sieve)

print("\n[+] prime generation")
for each in seven_bits_sieve_grammar.enumerate(length=7):
    print("[+] number generated", int(each[::-1], 2))

for i in range(2, 2**7):
    accepted = minimal_seven_sieve.read(get_binary_input(i))
    if i in sieve:
        assert accepted


[+] prime generation
[+] number generated 2
[+] number generated 3
[+] number generated 13
[+] number generated 19
[+] number generated 71
[+] number generated 23
[+] number generated 89
[+] number generated 115
[+] number generated 59
[+] number generated 55
[+] number generated 109
[+] number generated 21
[+] number generated 81
[+] number generated 25
[+] number generated 41
[+] number generated 43
[+] number generated 5
[+] number generated 113
[+] number generated 31
[+] number generated 77
[+] number generated 61
[+] number generated 127
[+] number generated 101
[+] number generated 95
[+] number generated 7
[+] number generated 117
[+] number generated 83
[+] number generated 107
[+] number generated 67
[+] number generated 53
[+] number generated 73
[+] number generated 37
[+] number generated 47
[+] number generated 79
[+] number generated 29
[+] number generated 11
[+] number generated 85
[+] number generated 97
[+] number generated 17
[+] number generated 103
[+] number gen

Indeed it is. We can compare the number of states and transitions for each automata in the building process.

In [45]:
def automata_info(name, fa: Automata):
    print("[+]", name, "states =", len(fa.transition.states))
    delta_function = fa.transition.transitions
    transitions = sum([len(delta_function[state]) for state in delta_function])
    print("[+]", name, "transitions =", transitions)
    print()

automata_info("NDFA 7 bits sieve", nfda_seven_sieve)
automata_info("DFA 7 bits sieve", dfa_seven_sieve)
automata_info("minimal DFA 7 bits sieve", minimal_seven_sieve)

[+] NDFA 7 bits sieve states = 144
[+] NDFA 7 bits sieve transitions = 171

[+] DFA 7 bits sieve states = 47
[+] DFA 7 bits sieve transitions = 64

[+] minimal DFA 7 bits sieve states = 26
[+] minimal DFA 7 bits sieve transitions = 43



The NDFA-DFA conversion / minimization process are working as a compression mechanism
to encode the binary representation of prime numbers up to a given $n \in \mathbb{N}$.

Regardless of how amusing might be to encode primes in this way, it would be interesting
to compare this technique with others. Let's push it forward and create a sieve up to $2^{16}$ for that.
The NDFA-DFA / minimization should be done in batches for a better scaling of the algorithms.

Before that, just realized that it's a bit annoying to write "NDFA-DFA / minimization process" each
time I have to reference it. And is not a very descriptive name. From now on, I'll call it
"the learning process", which is an universal process by which the number of states of the automata
is minimized without losing its correctness and the transitions are reorganized to support
this new structure.

The behavior has some similarities to neural networks, with the difference that the transitions (i.e
connections) of the automata might drastically change on each learning step. It's more evidence on
how powerful this type of architectures is when learning patterns in data.

As an example, take a look at what happens with even numbers, those with the least significant digit
equal to 0. They go through a state such that, if the next digit is not 1, the string is not accepted. In
other words, the automata learned that even numbers $>2$ are not primes, and is discarding them only with
2 transitions. This behavior also reminds me to some characteristic of decision trees.

Anyways, chit-chatting.

In [ ]:
import pickle

def fda_batch_update(max_value = 2**12):
    # 1000 new primes per batch
    batch_size = 750

    initial_state = State("ie0")
    final_state = State("ip0")

    builder = PrimeBuilder(final_state=final_state)

    transition = FADelta()
    final_set = {prime_state}

    runs = int(max_value / batch_size) + 1
    print("[+] total runs", runs)

    for j in range(0, runs):
        lower = j * batch_size + 1
        limit = min((j + 1) * batch_size, max_value + 1)

        print("[+] running batch", j, "between", lower, limit, "...")
        builder.build(initial_state, transition, lower, limit)

        # get NDFA and minimize it
        nfsm = FiniteAutomata(transition=transition, initial=initial_state, final=final_set)
        minimal = nfsm.minimal()

        # update final states
        final_set = minimal.final
        final_set.add(prime_state)

        # recover initial state
        initial_state = minimal.initial

        # get previous transition function
        transition = minimal.transition

    return minimal

max_sieve_value = 2**16
sixteen_sieve = fda_batch_update(max_value=max_sieve_value)

automata_info("minimal DFA 16 bits sieve", sixteen_sieve)
pickle.dump(sixteen_sieve, open("sixteen_sieve_minimal.pkl", "wb"))

for i in range(2, max_sieve_value):
    accepted = sixteen_sieve.read(get_binary_input(i))
    if i in sieve:
        assert accepted

[+] total runs 88
[+] running batch 0 between 1 750 ...
[+] running batch 1 between 751 1500 ...
[+] running batch 2 between 1501 2250 ...
[+] running batch 3 between 2251 3000 ...
[+] running batch 4 between 3001 3750 ...
[+] running batch 5 between 3751 4500 ...
[+] running batch 6 between 4501 5250 ...
[+] running batch 7 between 5251 6000 ...
[+] running batch 8 between 6001 6750 ...
[+] running batch 9 between 6751 7500 ...
[+] running batch 10 between 7501 8250 ...
[+] running batch 11 between 8251 9000 ...
[+] running batch 12 between 9001 9750 ...
[+] running batch 13 between 9751 10500 ...
[+] running batch 14 between 10501 11250 ...
[+] running batch 15 between 11251 12000 ...
[+] running batch 16 between 12001 12750 ...
[+] running batch 17 between 12751 13500 ...
[+] running batch 18 between 13501 14250 ...
[+] running batch 19 between 14251 15000 ...
[+] running batch 20 between 15001 15750 ...
[+] running batch 21 between 15751 16500 ...
[+] running batch 22 between 16501